In [1]:
import os
import torch
import numpy as np
import torch.nn as nn
from google.colab import drive
from matplotlib import pyplot as plt
from torchvision import datasets ,transforms, utils
from torch.utils.data.dataset import Dataset 
from torch.utils.data import Dataset , DataLoader
from PIL import Image
import csv

In [2]:
#獲得路徑,位置list並存入dic
drive.mount('/content/drive')
num=20
file_name = [str(i)+'.jpg' for i in range(num)]
file_path=[os.path.join('/content/drive/MyDrive/DL_final/puzzle_train/orig_val/',i)for i in file_name]
pos_numpy=np.loadtxt('/content/drive/MyDrive/DL_final/puzzle_train/pos.txt',dtype=str)
pos_list =list(pos_numpy)
dis={}
for i in range(len(pos_list)):
  dis[pos_list[i]]=i
preporecess_mean= transforms.Compose([transforms.Resize([224,224]),transforms.ToTensor()])
imgs=preporecess_mean(Image.open(file_path[1]))
imgs.shape

Mounted at /content/drive


torch.Size([3, 224, 224])

In [3]:
#將圖片切割pos_num輸入是0,1,2,3分別對應圖片四角，回傳對應位置切割好的圖片
def image_cut(imgs,pos_num):
  pos_num=pos_num
  if pos_num==0:
    imgs=imgs.crop((0,0,imgs.size[0]/2,imgs.size[1]/2))
  elif pos_num==1:
    imgs=imgs.crop((imgs.size[0]/2,0,imgs.size[0],imgs.size[1]/2))
  elif pos_num==2:
    imgs=imgs.crop((0,imgs.size[1]/2,imgs.size[0]/2,imgs.size[1]))
  elif pos_num==3:
    imgs=imgs.crop((imgs.size[0]/2,imgs.size[1]/2,imgs.size[0],imgs.size[1]))
  else:
    print("xx")
  return imgs 
#將圖片切割後根據輸入黏回，輸入是0,1,2,3分別對應圖片四角，回傳對應位置黏好的圖片
def image_paste(image_result,imgs,pos_num):
  x=int(image_result.size[0]/2)
  y=int(image_result.size[1]/2)
  if pos_num==0:
    image_result.paste(imgs,(0,0))
  elif pos_num==1:
    image_result.paste(imgs,(x,0))
  elif pos_num==2:
    image_result.paste(imgs,(0,y))
  elif pos_num==3:
    image_result.paste(imgs,(x,y))
  else:
    print("xx")
  return image_result

In [4]:
labels = {}
for i in range(num):        #輸入幾張原圖
  randArray = np.random.choice(24, 5, replace=False)
  for j in range(len(randArray)):   #用一張圓圖做出幾張亂圖
    image_result=Image.new('RGB',(224,224))
    for k in range(4):    #將圖形根據pos_numpy出來的數值打亂並重黏出一張新圖
      img_cut=image_cut(Image.open(file_path[i]).resize((224,224)),int(pos_numpy[randArray[j]][k]))
      image_result=image_paste(image_result,img_cut,k)
    image_result.save('/content/drive/MyDrive/DL_final/puzzle_train/validation/'+str(i)+'_'+str(j)+'.jpg')
    labels[str(i)+'_'+str(j)] = pos_numpy[randArray[j]]

In [5]:
with open('/content/drive/MyDrive/DL_final/puzzle_train/val_labels.csv', 'w') as f:
  writer = csv.writer(f)
  for k, v in labels.items():
    writer.writerow([k, v])

In [6]:
#將圖形根據pos_numpy出來的數值打亂並重黏出一張新圖
image_result=Image.new('RGB',(int(Image.open(file_path[5]).size[0]),int(Image.open(file_path[5]).size[1])))
for i in range(4):  
  img_cut=image_cut(Image.open(file_path[6]).resize((224,224)),int(pos_numpy[6][i]))
  image_result=image_paste(image_result,img_cut,i)
image_result.size

(1200, 801)

In [7]:
labels
pos_list

['0123',
 '0132',
 '0213',
 '0231',
 '0312',
 '0321',
 '1023',
 '1032',
 '1203',
 '1230',
 '1302',
 '1320',
 '2013',
 '2031',
 '2103',
 '2130',
 '2301',
 '2310',
 '3012',
 '3021',
 '3102',
 '3120',
 '3201',
 '3210']